In [22]:
import numpy as np
import pandas as pd
import os
import sklearn as sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Reading Data

In [23]:
def segmentWords(s): 
    return s.split()

def readFile(fileName):
    # Function for reading file
    # input: filename as string
    # output: contents of file as list containing single words
    contents = []
    f = open(fileName)
    for line in f:
        contents.append(line)
    f.close()
    result = segmentWords('\n'.join(contents))
    return result

In [24]:
for c in os.listdir("data_training"):
    directory = "data_training/" + c
    for file in os.listdir(directory):
        print(file)
        words = readFile(directory + "/" + file)
        print(len(words))
        break

cv000_29416.txt
825
cv000_29590.txt
802


In [25]:
os.listdir("data_training")

['neg', 'pos']

#### Create a Dataframe containing the counts of each word in a file

In [26]:
d = []

for c in os.listdir("data_training"):
    directory = "data_training/" + c
    for file in os.listdir(directory):
        words = readFile(directory + "/" + file)
        e = {x:words.count(x) for x in words}
        #e['__FileID__'] = file
        e['__CLASS__'] = c
        d.append(e)

Create a dataframe from d - make sure to fill all the nan values with zeros.

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html


In [27]:
df = pd.DataFrame(d)
dataframe = df.fillna(0)

#### Split data into training and validation set 

* Sample 80% of your dataframe to be the training data

* Let the remaining 20% be the validation data (you can filter out the indicies of the original dataframe that weren't selected for the training data)

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [28]:
training_data = dataframe.sample(frac = 0.8)#.drop(labels = ['__FileID__'],axis = 1)
validation_data = dataframe.drop(labels =training_data.index, axis = 0)#.drop(labels = ['__CLASS__'],axis = 1)


#training_data,validation_data = train_test_split(dataframe.values, test_size=0.2)

* Split the dataframe for both training and validation data into x and y dataframes - where y contains the labels and x contains the words

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [29]:
x_training_data = training_data.drop(labels = '__CLASS__',axis = 1)
y_training_data = training_data['__CLASS__']
x_validation_data = validation_data.drop(labels = '__CLASS__',axis = 1)
y_validation_data = validation_data['__CLASS__']

# Logistic Regression

#### Basic Logistic Regression
* Use sklearn's linear_model.LogisticRegression() to create your model.
* Fit the data and labels with your model.
* Score your model with the same data and labels.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [30]:
#using default regularization term
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
my_logistic_reg = logistic.fit(x_training_data,y_training_data)
print('Score of Logistic Regression using default regularization term: ',my_logistic_reg.score(x_validation_data,y_validation_data))

Score of Logistic Regression using default regularization term:  0.840625


I am not overfitting, a score of 0.85 is really good

#### Changing Parameters

In [31]:
#using customized regularization term and methodology
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(C = 1e6)
my_logistic_reg = logistic.fit(x_training_data,y_training_data)
print('Score of Logistic Regression using customized regularization term and newton-cg method: ',my_logistic_reg.score(x_validation_data,y_validation_data))


Score of Logistic Regression using customized regularization term and newton-cg method:  0.834375


#### Feature Selection
* In the backward stepsize selection method, you can remove coefficients and the corresponding x columns, where the coefficient is more than a particular amount away from the mean - you can choose how far from the mean is reasonable.

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.std.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.mean.html

How did you select which features to remove? Why did that reduce overfitting?

In [32]:
np.std(x_training_data)

               0.027940
earth          0.027940
goodies        0.027940
if             0.027940
ripley         0.027940
suspend        0.027940
they           0.027940
white         0.027940
               0.083762
               0.284890
!               2.001591
"              10.548344
#               0.027940
#05425          0.027940
#1              0.092435
#13             0.000000
#2              0.083674
#3              0.027940
#5              0.027940
#6              0.027940
#8              0.027940
#9              0.027940
#@%$^           0.027940
$1              0.055814
$10             0.088167
$100            0.088042
$1000           0.039498
$100m           0.000000
$11             0.027940
$115            0.000000
                 ...    
zoo-like        0.000000
zookeeper       0.027940
zoolander       0.000000
zoolander's     0.000000
zoologist       0.048356
zoom            0.027940
zoom-in's       0.027940
zoom-ins        0.000000
zoom-out        0.027940


In [34]:
np.std(np.mean(x_training_data))

0.3640881445395468

In [39]:
n_col = {}
for col_name in x_training_data:
    temp = np.mean(x_training_data[col_name])
    if temp > .02:
        n_col[col_name] = x_training_data[col_name]

In [41]:
x_training_data.shape

(1280, 45671)

In [40]:
pd.DataFrame.from_dict(n_col).shape

(1280, 3090)

# Single Decision Tree

#### Basic Decision Tree

* Initialize your model as a decision tree with sklearn.
* Fit the data and labels to the model.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html


In [11]:
clf = DecisionTreeClassifier(random_state=0)
decision_tree = clf.fit(x_training_data,y_training_data)
print('The accuracy of default tree is:', decision_tree.score(x_validation_data,y_validation_data))

The accuracy of default tree is: 0.628125


#### Changing Parameters
* To test out which value is optimal for a particular parameter, you can either loop through various values or look into sklearn.model_selection.GridSearchCV

References:


http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [12]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
svc = svm.SVC(C=1, kernel='linear')
svc_fit = svc.fit(x_training_data,y_training_data)
svc_score = svc_fit.score(x_validation_data,y_validation_data)
print(svc_score)

0.81875


In [13]:
new_fit = svm.SVC(C=10, kernel='rbf', gamma=0.001).fit(x_training_data,y_training_data)

In [14]:
new_fit.score(x_validation_data,y_validation_data)

0.72187500000000004

How did you choose which parameters to change and what value to give to them? Feel free to show a plot.

Why is a single decision tree so prone to overfitting?

A single decision tree is prone to overfitting because the data we put in is random and contain many noise. By executing multiple trials over different trees (of the same data), we can ensure to reduce those noise and produce the true result.  

# Random Forest Classifier

#### Basic Random Forest

* Use sklearn's ensemble.RandomForestClassifier() to create your model.
* Fit the data and labels with your model.
* Score your model with the same data and labels.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


In [15]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
my_forest = clf.fit(x_training_data,y_training_data)
my_forest.score(x_validation_data,y_validation_data)

0.56874999999999998

#### Changing Parameters

What parameters did you choose to change and why?

How does a random forest classifier prevent overfitting better than a single decision tree?